In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import os
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform
from six import BytesIO
import tensorflow as tf
import tensorflow_hub as hub
from six.moves.urllib.request import urlopen

In [2]:
tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.FATAL)

In [3]:
trainpath = './customdata/ResizedTrain/'
valdir = './customdata/ResizedVal/'

building_descs = []

In [4]:
m = hub.Module('https://tfhub.dev/google/delf/1')

In [5]:
#Alternate method

# img_name = []

# for i in os.listdir(path):
#     img_name.append('./datacluster/Train/'+i)

# print img_name

In [6]:
# def resize_image(srcfile, destfile, new_width=256, new_height=256):
#     pil_image = Image.open(srcfile)
#     pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
#     pil_image_rgb = pil_image.convert('RGB')
#     pil_image_rgb.save(destfile, format='JPEG', quality=90)
#     return destfile
# def resize_images_folder(srcfolder, destfolder='./datacluster/ResizedVal/', new_width=256, new_height=256):
#     for srcfile in glob.iglob(os.path.join('./datacluster/Val/', '*.[Jj][Pp][Ee][Gg]')):
#         src_basename = os.path.basename(srcfile)
#         destfile=os.path.join(destfolder,src_basename)
#         resize_image(srcfile, destfile, new_width, new_height)
#     return destfolder

def get_resized_db_image_paths(destfolder=trainpath):
    return sorted(list(glob.iglob(os.path.join(destfolder, '*.[Jj][Pp][Ee][Gg]'))))

In [7]:
# resize_images_folder('./datacluster/Train')

In [8]:
# resize_images_folder(path)
db_images = get_resized_db_image_paths()
print (db_images)

['./customdata/ResizedTrain/Buddhist_1.jpeg', './customdata/ResizedTrain/Buddhist_2.jpeg', './customdata/ResizedTrain/Buddhist_4.jpeg', './customdata/ResizedTrain/Dravidian_1.jpeg', './customdata/ResizedTrain/Dravidian_3.jpeg', './customdata/ResizedTrain/Dravidian_6.jpeg', './customdata/ResizedTrain/Kalinga_75.jpeg', './customdata/ResizedTrain/Kalinga_77.jpeg', './customdata/ResizedTrain/Kalinga_79.jpeg', './customdata/ResizedTrain/Mughal_84.jpeg', './customdata/ResizedTrain/Mughal_86.jpeg', './customdata/ResizedTrain/Mughal_94.jpeg']


In [9]:
import operator

def accumulate(inputs, func):
    itr = iter(inputs)
    prev = next(itr)
    for cur in itr:
        yield prev
        prev = func(prev, cur)

In [10]:
image_placeholder = tf.placeholder(
    tf.float32, shape=(None, None, 3), name='input_image')

module_inputs = {
    'image': image_placeholder,
    'score_threshold': 100.0,
    'image_scales': [0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0],
    'max_feature_num': 1000,
}

In [11]:
#number_of_files = len([item for item in os.listdir(path) if os.path.isfile(os.path.join(path, item))])

In [12]:
def image_input_fn(image_files):
    filename_queue = tf.train.string_input_producer(
        image_files, shuffle=False)
    reader = tf.WholeFileReader()
    _, value = reader.read(filename_queue)
    image_tf = tf.image.decode_jpeg(value, channels=3)
    return tf.image.convert_image_dtype(image_tf, tf.float32)

In [13]:
module_outputs = m(module_inputs, as_dict=True)

image_tf = image_input_fn(db_images)

with tf.train.MonitoredSession() as sess:
    results_dict = {}  # Stores the locations and their descriptors for each image
    for image_path in db_images:
        image = sess.run(image_tf)
        print('Extracting locations and descriptors from %s' % image_path)
        building_descs.append(image_path.split('_')[0].split('/')[-1])
        results_dict[image_path] = sess.run(
            [module_outputs['locations'], module_outputs['descriptors']],
            feed_dict={image_placeholder: image})

Extracting locations and descriptors from ./customdata/ResizedTrain/Buddhist_1.jpeg
Extracting locations and descriptors from ./customdata/ResizedTrain/Buddhist_2.jpeg
Extracting locations and descriptors from ./customdata/ResizedTrain/Buddhist_4.jpeg
Extracting locations and descriptors from ./customdata/ResizedTrain/Dravidian_1.jpeg
Extracting locations and descriptors from ./customdata/ResizedTrain/Dravidian_3.jpeg
Extracting locations and descriptors from ./customdata/ResizedTrain/Dravidian_6.jpeg
Extracting locations and descriptors from ./customdata/ResizedTrain/Kalinga_75.jpeg
Extracting locations and descriptors from ./customdata/ResizedTrain/Kalinga_77.jpeg
Extracting locations and descriptors from ./customdata/ResizedTrain/Kalinga_79.jpeg
Extracting locations and descriptors from ./customdata/ResizedTrain/Mughal_84.jpeg
Extracting locations and descriptors from ./customdata/ResizedTrain/Mughal_86.jpeg
Extracting locations and descriptors from ./customdata/ResizedTrain/Mughal_

In [15]:
locations_agg = np.concatenate([results_dict[img][0] for img in db_images])
descriptors_agg = np.concatenate([results_dict[img][1] for img in db_images])
accumulated_indexes_boundaries = list(accumulate([results_dict[img][0].shape[0] for img in db_images], operator.add))

d_tree = cKDTree(descriptors_agg) # build the KD tree

In [29]:
print (accumulated_indexes_boundaries)

[205, 325, 518, 699, 944, 1129, 1316, 1599, 1850, 2092, 2325]


In [28]:
def compute_locations_and_descriptors(image_path):
    tf.reset_default_graph()
    tf.logging.set_verbosity(tf.logging.FATAL)

    m = hub.Module('https://tfhub.dev/google/delf/1')

    # The module operates on a single image at a time, so define a placeholder to
    # feed an arbitrary image in.
    image_placeholder = tf.placeholder(
        tf.float32, shape=(None, None, 3), name='input_image')

    module_inputs = {
        'image': image_placeholder,
        'score_threshold': 100.0,
        'image_scales': [0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0],
        'max_feature_num': 1000,
    }

    module_outputs = m(module_inputs, as_dict=True)

    image_tf = image_input_fn([image_path])

    with tf.train.MonitoredSession() as sess:
        image = sess.run(image_tf)
        print('Extracting locations and descriptors from %s' % image_path)
        return sess.run(
            [module_outputs['locations'], module_outputs['descriptors']],
            feed_dict={image_placeholder: image})

In [24]:
def image_index_2_accumulated_indexes(index, accumulated_indexes_boundaries):
    '''
    Image index to accumulated/aggregated locations/descriptors pair indexes.
    '''
    if index > len(accumulated_indexes_boundaries) - 1:
        return None
    accumulated_index_start = None
    accumulated_index_end = None
    if index == 0:
        accumulated_index_start = 0
        accumulated_index_end = accumulated_indexes_boundaries[index]
    else:
        accumulated_index_start = accumulated_indexes_boundaries[index-1]
        accumulated_index_end = accumulated_indexes_boundaries[index]
    return np.arange(accumulated_index_start,accumulated_index_end)

In [25]:
def get_locations_2_use(image_db_index, k_nearest_indices, accumulated_indexes_boundaries):
    '''
    Get a pair of locations to use, the query image to the database image with given index.
    Return: a tuple of 2 numpy arrays, the locations pair.
    '''
    image_accumulated_indexes = image_index_2_accumulated_indexes(image_db_index, accumulated_indexes_boundaries)
    locations_2_use_query = []
    locations_2_use_db = []
    for i, row in enumerate(k_nearest_indices):
        for acc_index in row:
            if acc_index in image_accumulated_indexes:
                locations_2_use_query.append(query_image_locations[i])
                locations_2_use_db.append(locations_agg[acc_index])
                break
    return np.array(locations_2_use_query), np.array(locations_2_use_db)
            

In [26]:
count = 0
tot = 0
distance_threshold = 0.8
# K nearest neighbors
K = 10

In [32]:
for im in os.listdir(valdir):
    
    query_image_locations, query_image_descriptors = compute_locations_and_descriptors(valdir+im)
    distances, indices = d_tree.query(
    query_image_descriptors, distance_upper_bound=distance_threshold, k = K, n_jobs=-1)

    # Find the list of unique accumulated/aggregated indexes
    unique_indices = np.array(list(set(indices.flatten())))

    unique_indices.sort()
    if unique_indices[-1] == descriptors_agg.shape[0]:
        unique_indices = unique_indices[:-1]
        
    unique_image_indexes = np.array(
    list(set([np.argmax([np.array(accumulated_indexes_boundaries)>index]) 
              for index in unique_indices])))
    unique_image_indexes
    
    inliers_counts = []
    
    img_1 = mpimg.imread(valdir+im)
    for index in unique_image_indexes:
        locations_2_use_query, locations_2_use_db = get_locations_2_use(index, indices, accumulated_indexes_boundaries)
        # Perform geometric verification using RANSAC.
        _, inliers = ransac(
            (locations_2_use_db, locations_2_use_query), # source and destination coordinates
            AffineTransform,
            min_samples=3,
            residual_threshold=20,
            max_trials=1000)
        # If no inlier is found for a database candidate image, we continue on to the next one.
        if inliers is None or len(inliers) == 0:
            continue
        # the number of inliers as the score for retrieved images.
        inliers_counts.append({"index": index, "inliers": sum(inliers)})
        print('Found inliers for image {} -> {}'.format(index, sum(inliers)))
    
    print (inliers_counts)
    
    top_match = sorted(inliers_counts, key=lambda k: k['inliers'], reverse=True)[0]

    index = top_match['index']
    print ('Best guess for this image:', building_descs[index])
    if im.split('_')[0]== building_descs[index]:
        count = count+1
        
    locations_2_use_query, locations_2_use_db = get_locations_2_use(index, indices, accumulated_indexes_boundaries)
    # Perform geometric verification using RANSAC.
    _, inliers = ransac(
        (locations_2_use_db, locations_2_use_query), # source and destination coordinates
        AffineTransform,
        min_samples=3,
        residual_threshold=20,
        max_trials=1000)
    tot= tot+1
    
print ('Accuracy:', count/tot)

Extracting locations and descriptors from ./customdata/ResizedVal/Buddhist_727.jpeg
Found inliers for image 0 -> 10


/home/sagnik/.local/lib/python3.5/site-packages/skimage/transform/_geometric.py:684: RuntimeWarning: divide by zero encountered in true_divide
  H.flat[list(self._coeffs) + [8]] = - V[-1, :-1] / V[-1, -1]
/home/sagnik/.local/lib/python3.5/site-packages/skimage/transform/_geometric.py:684: RuntimeWarning: invalid value encountered in true_divide
  H.flat[list(self._coeffs) + [8]] = - V[-1, :-1] / V[-1, -1]
/home/sagnik/.local/lib/python3.5/site-packages/skimage/measure/fit.py:831: RuntimeWarning: invalid value encountered in less
  sample_model_inliers = sample_model_residuals < residual_threshold


Found inliers for image 1 -> 3
Found inliers for image 2 -> 21
Found inliers for image 4 -> 2
Found inliers for image 6 -> 6
Found inliers for image 7 -> 2
Found inliers for image 8 -> 2
Found inliers for image 9 -> 6
Found inliers for image 10 -> 8
[{'index': 0, 'inliers': 10}, {'index': 1, 'inliers': 3}, {'index': 2, 'inliers': 21}, {'index': 4, 'inliers': 2}, {'index': 6, 'inliers': 6}, {'index': 7, 'inliers': 2}, {'index': 8, 'inliers': 2}, {'index': 9, 'inliers': 6}, {'index': 10, 'inliers': 8}]
Best guess for this image: Buddhist
Extracting locations and descriptors from ./customdata/ResizedVal/Dravidian_691.jpeg
Found inliers for image 0 -> 4
Found inliers for image 2 -> 9
Found inliers for image 3 -> 11
Found inliers for image 4 -> 16
Found inliers for image 5 -> 8
Found inliers for image 6 -> 8
Found inliers for image 7 -> 13
Found inliers for image 8 -> 6
Found inliers for image 9 -> 3
Found inliers for image 10 -> 9
[{'index': 0, 'inliers': 4}, {'index': 2, 'inliers': 9}, {'